<a href="https://colab.research.google.com/github/trevorwjames/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/Trevor_James_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [372]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [373]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [374]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [375]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [376]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23040 entries, 0 to 23039
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         23040 non-null  object 
 1   NEIGHBORHOOD                    23040 non-null  object 
 2   BUILDING_CLASS_CATEGORY         23040 non-null  object 
 3   TAX_CLASS_AT_PRESENT            23039 non-null  object 
 4   BLOCK                           23040 non-null  int64  
 5   LOT                             23040 non-null  int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       23039 non-null  object 
 8   ADDRESS                         23040 non-null  object 
 9   APARTMENT_NUMBER                5201 non-null   object 
 10  ZIP_CODE                        23039 non-null  float64
 11  RESIDENTIAL_UNITS               23039 non-null  float64
 12  COMMERCIAL_UNITS                

In [377]:
df['BUILDING_CLASS_CATEGORY']

0          13 CONDOS - ELEVATOR APARTMENTS
1                      21 OFFICE BUILDINGS
2                      21 OFFICE BUILDINGS
3          13 CONDOS - ELEVATOR APARTMENTS
4        15 CONDOS - 2-10 UNIT RESIDENTIAL
                       ...                
23035              01 ONE FAMILY DWELLINGS
23036         09 COOPS - WALKUP APARTMENTS
23037       10 COOPS - ELEVATOR APARTMENTS
23038              02 TWO FAMILY DWELLINGS
23039        12 CONDOS - WALKUP APARTMENTS
Name: BUILDING_CLASS_CATEGORY, Length: 23040, dtype: object

In [378]:
# subsettig data to instructions 

# Only looking at single family dwellings
subset = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']

# nothing under 100k
subset = subset[subset['SALE_PRICE'] > 100000]

#nothing over 2mil
subset = subset[subset['SALE_PRICE'] < 2000000]

subset['SALE_PRICE'].describe()

count    3.151000e+03
mean     6.285601e+05
std      2.929904e+05
min      1.040000e+05
25%      4.475000e+05
50%      5.680000e+05
75%      7.600000e+05
max      1.955000e+06
Name: SALE_PRICE, dtype: float64

In [379]:
subset.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [380]:
# creating train/test split
train = subset[subset['SALE_DATE'] < '04/01/2019']
test = subset[subset['SALE_DATE'] > '04/01/2019']

In [381]:
train['SALE_DATE'].tail()

18129    03/29/2019
18130    03/29/2019
18132    03/29/2019
18134    03/29/2019
18147    03/30/2019
Name: SALE_DATE, dtype: object

In [382]:
test['SALE_DATE'].tail()

23029    04/30/2019
23031    04/30/2019
23032    04/30/2019
23033    04/30/2019
23035    04/30/2019
Name: SALE_DATE, dtype: object

In [383]:
# Target variable
y_train = train['SALE_PRICE']
y_test = test['SALE_PRICE']

print(y_test.shape)
y_train.shape

(603,)


(2507,)

In [384]:
train['NEIGHBORHOOD'].value_counts()

OTHER                 2382
FLUSHING-NORTH          77
FOREST HILLS            17
BOROUGH PARK            12
ASTORIA                 11
BEDFORD STUYVESANT       8
Name: NEIGHBORHOOD, dtype: int64

In [385]:
X_train = train.drop(['EASE-MENT','ADDRESS', 'APARTMENT_NUMBER', 'BUILDING_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_TIME_OF_SALE', 'NEIGHBORHOOD', 'SALE_PRICE', 'LAND_SQUARE_FEET', "SALE_DATE"], axis=1)
X_test = test.drop(['EASE-MENT', 'ADDRESS', 'APARTMENT_NUMBER', 'BUILDING_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_TIME_OF_SALE', 'NEIGHBORHOOD', 'SALE_PRICE', 'LAND_SQUARE_FEET', "SALE_DATE"], axis=1)

In [386]:
X_train.head()

,BOROUGH,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
44,3,01 ONE FAMILY DWELLINGS,1,5495,801,11230.0,1.0,0.0,1.0,1325.0,1930.0,1
61,4,01 ONE FAMILY DWELLINGS,1,7918,72,11427.0,1.0,0.0,1.0,2001.0,1940.0,1
78,2,01 ONE FAMILY DWELLINGS,1,4210,19,10461.0,1.0,0.0,1.0,2043.0,1925.0,1
108,3,01 ONE FAMILY DWELLINGS,1,5212,69,11226.0,1.0,0.0,1.0,2680.0,1899.0,1
111,3,01 ONE FAMILY DWELLINGS,1,7930,121,11203.0,1.0,0.0,1.0,1872.0,1940.0,1


In [387]:
# OHE import and instantiate
from category_encoders import OneHotEncoder

ohe = OneHotEncoder(use_cat_names=True)

In [388]:
# Running ohe on the newly formed subset of data
X_train_trans = ohe.fit_transform(X_train)

In [389]:
# OHE of boroughs in order to run a better model that can see relationships on boroughs
# Not sure how to add this to the subset to run kbest with this here
X_train_trans = pd.DataFrame(X_train_trans)
X_train_trans.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
44,1,0,0,0,0,1,1,0,5495,801,11230.0,1.0,0.0,1.0,1325.0,1930.0,1
61,0,1,0,0,0,1,1,0,7918,72,11427.0,1.0,0.0,1.0,2001.0,1940.0,1
78,0,0,1,0,0,1,1,0,4210,19,10461.0,1.0,0.0,1.0,2043.0,1925.0,1
108,1,0,0,0,0,1,1,0,5212,69,11226.0,1.0,0.0,1.0,2680.0,1899.0,1
111,1,0,0,0,0,1,1,0,7930,121,11203.0,1.0,0.0,1.0,1872.0,1940.0,1


In [390]:
X_test_trans = ohe.transform(X_test)

X_test_trans.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
18500,0,0,1,0,0,1,1,0,2555,25,10454.0,1.0,0.0,1.0,1104.0,2000.0,1
18520,1,0,0,0,0,1,1,0,5575,27,11204.0,1.0,0.0,1.0,3690.0,1920.0,1
18521,1,0,0,0,0,1,1,0,6293,29,11228.0,1.0,0.0,1.0,1296.0,1930.0,1
18525,1,0,0,0,0,1,1,0,8448,68,11234.0,1.0,0.0,1.0,1665.0,1970.0,1
18549,1,0,0,0,0,1,1,0,7017,110,11224.0,1.0,0.0,1.0,1344.0,1993.0,1


In [391]:
X_trainnum = X_train_trans.select_dtypes(include='number')
X_testnum = X_test_trans.select_dtypes(include='number')

In [392]:
X_testnum.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
18500,0,0,1,0,0,1,1,0,2555,25,10454.0,1.0,0.0,1.0,1104.0,2000.0,1
18520,1,0,0,0,0,1,1,0,5575,27,11204.0,1.0,0.0,1.0,3690.0,1920.0,1
18521,1,0,0,0,0,1,1,0,6293,29,11228.0,1.0,0.0,1.0,1296.0,1930.0,1
18525,1,0,0,0,0,1,1,0,8448,68,11234.0,1.0,0.0,1.0,1665.0,1970.0,1
18549,1,0,0,0,0,1,1,0,7017,110,11224.0,1.0,0.0,1.0,1344.0,1993.0,1


In [393]:
X_trainnum.shape

(2507, 17)

In [394]:
y_train.shape

(2507,)

In [395]:
from sklearn.feature_selection import SelectKBest
selector = SelectKBest(k=10)

X_train_selected = selector.fit_transform(X_trainnum, y_train)

X_test_selected = selector.transform(X_testnum)

#Runtime warning NaN in DF?

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 4  5 16] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [396]:
mask = selector.get_support()

In [397]:
X_trainnum.columns[mask]

Index(['BOROUGH_1', 'TAX_CLASS_AT_PRESENT_1', 'TAX_CLASS_AT_PRESENT_1D', 'LOT',
       'ZIP_CODE', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT'],
      dtype='object')

In [398]:
# Ridge Regression with Kbest columns
from sklearn.linear_model import Ridge

In [399]:
# instantiate 
ridge = Ridge(normalize=True)

In [400]:
ridge.fit(X_trainnum, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None, normalize=True,
      random_state=None, solver='auto', tol=0.001)

In [401]:
ridge.coef_

array([ 7.08014637e+04,  2.47053909e+04, -7.41190590e+04, -4.72517161e+04,
        6.06805924e+04,  0.00000000e+00, -4.48520430e+03,  4.48520430e+03,
       -8.62503123e+00, -2.80558937e+01,  4.75044806e+01, -4.48520430e+03,
        6.27386109e+04,  3.40503891e+04,  1.06982120e+02,  6.26520446e+01,
        0.00000000e+00])

In [402]:
# inspecting some metrics from the test data set that has run through the model
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

y_pred = ridge.predict(X_testnum)

mae = mean_absolute_error(y_test, y_pred)

print('Mean Absolute Error for test data set: $',mae.round(0))

Mean Absolute Error for test data set: $ 180199.0


In [403]:
# Mean squared error
mse = mean_squared_error(y_test, y_pred)

print('Mean Squared Error for test data set: $',mse.round(0))

Mean Squared Error for test data set: $ 68111405708.0


In [404]:
# r2 score

r2 = ridge.score(X_testnum, y_test)

r2

0.22504487464853584

In [408]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X_trainnum, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [409]:
y_predlr = lr.predict(X_testnum)

In [410]:
mean_absolute_error(y_test, y_predlr)

169183.77128193967